In [1]:
from googlesearch import search
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to perform Google search and get links
def get_search_results(query, num_results=2):
    search_results = search(query, num_results=num_results)
    return list(search_results)

# Function to extract content from a search result
def extract_content_from_url(url, timeout=10):
    try:
        response = requests.get(url, timeout=timeout)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            # Extract all text content from the page
            return soup.get_text(separator="\n")
        else:
            return f"Failed to retrieve webpage. Status code: {response.status_code}"
    except requests.exceptions.Timeout:
        return "Request timed out"
    except requests.exceptions.RequestException as e:
        return f"Request failed: {e}"

# Function to search for specific terms in the content
def search_for_terms(content, terms):
    for term in terms:
        if term.lower() in content.lower():
            return True
    return False

# Read symbols, dates, and split ratios from the CSV file
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/DiscordBot/RevSplitBotVersion/AlphaPRIABot/TestData.csv'
df_symbols = pd.read_csv(input_file_path)
symbols = df_symbols['Symbol'].tolist()
dates = df_symbols['Date'].tolist()
split_ratios = df_symbols['Split Ratio'].tolist()

# Terms to search for very basic
terms = [" round ", " rounded ", " rounding "]

# Prepare the result list
results = []

# Process each symbol, its corresponding date, and split ratio
for symbol, date, split_ratio in zip(symbols, dates, split_ratios):
    query = f"{symbol} Reverse split press release"
    print(f"\nSearching for: {query}")

    # Get the top 2 search results
    links = get_search_results(query, num_results=1)
    
    # Flag to track if "DO BUY" should be set
    do_buy_flag = False

    # Extract and check content from the search results
    for i, link in enumerate(links):
        print(f"\nExtracting content from search result {i+1}: {link}")
        content = extract_content_from_url(link)
        
        # Check for specific terms in the content
        if search_for_terms(content, terms):
            do_buy_flag = True
            break

    # Determine final decision
    decision = "ROUNDING" if do_buy_flag else "FORGO"
    results.append([date, symbol, decision, split_ratio])
    print(f"Decision for {symbol}: {decision}")

# Create a DataFrame from the results
result_df = pd.DataFrame(results, columns=['Date', 'Symbol', 'Decision', 'Split Ratio'])

# Save the results to a new CSV file
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/DiscordBot/RevSplitBotVersion/AlphaPRIABot/Results.csv'
result_df.to_csv(output_file_path, index=False)

print(f"\nResults have been saved to {output_file_path}")


/Users/ardadinc/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(



Searching for: IMCC Reverse split press release

Extracting content from search result 1: https://www.nasdaqtrader.com/TraderNews.aspx?id=ECA2024-336

Extracting content from search result 2: https://www.streetinsider.com/Corporate+News/IM+Cannabis+Corp+%28IMCC%29+Completes+6%3A1+Reverse+Share+Split/23457880.html

Extracting content from search result 3: https://www.prnewswire.com/news-releases/im-cannabis-common-shares-commence-trading-on-consolidated-basis-after-1-for-10-reverse-stock-split-301681378.html
Decision for IMCC: ROUNDING

Searching for: NYCB Reverse split press release

Extracting content from search result 1: https://ir.mynycb.com/news-and-events/news-releases/press-release-details/2024/NEW-YORK-COMMUNITY-BANCORP-INC.-ANNOUNCES-EFFECTIVENESS-OF-ONE-FOR-THREE-REVERSE-STOCK-SPLIT/default.aspx

Extracting content from search result 2: https://www.prnewswire.com/news-releases/new-york-community-bancorp-inc-announces-effectiveness-of-one-for-three-reverse-stock-split-3021952

<h2>-----------------------------Add Price Data to RevSplits-----------------------------<h2>

In [2]:
import pandas as pd
import yfinance as yf

# Path to the input CSV file (X.csv)
input_file_path = '/Users/ardadinc/Desktop/Market-Insight/DiscordBot/RevSplitBotVersion/AlphaPRIABot/Results.csv'
# Path to the output CSV file (Y.csv)
output_file_path = '/Users/ardadinc/Desktop/Market-Insight/DiscordBot/RevSplitBotVersion/AlphaPRIABot/Results.csv'

# Read the input CSV file
df = pd.read_csv(input_file_path)

# Remove rows where the 'Decision' column has the value 'FORGO'
df = df[df['Decision'] != 'FORGO']

# Function to get the latest stock price from yfinance and round to 3 decimals
def get_latest_stock_price(symbol):
    try:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="1d")
        if not hist.empty:
            return round(hist['Close'].iloc[0], 2)
        else:
            return None
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# Add a new column for the stock price
df['Price'] = df['Symbol'].apply(get_latest_stock_price)

# Save the updated DataFrame to the output CSV file
df.to_csv(output_file_path, index=False)

print(f"Updated data with stock prices has been saved to {output_file_path}")


Updated data with stock prices has been saved to /Users/ardadinc/Desktop/Market-Insight/DiscordBot/RevSplitBotVersion/AlphaPRIABot/Results.csv
